#Телеграм-бот для генерации стихотворений

для запуска бота просто запустить все ячейки

In [8]:
#!pip install python-telegram-bot

In [15]:
#!pip install dialogflow

In [1]:
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import logging
from telegram import Update

In [2]:
#Настройки
updater = Updater(token='179767919:AAFB8nPqel_MhstnubHo0Qs1T9c_8yQ5Wo') # Токен API к Telegram
dispatcher = updater.dispatcher

In [3]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import transformers
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# предварительно нужно скачать модели
#модель для генерации стихов Есенина
#https://drive.google.com/drive/folders/1iCKCxtkIrjl-IAEswTUdgavqEMNOPylU?usp=sharing
#модель для генерации стихов Лермонтова
#https://drive.google.com/drive/folders/11cuy0YTItu_RdHi75X2Vh4F-Pm7pe54R?usp=sharing
#
#

In [5]:
def generate_lerm(prefix):
    #загружаем модель
    model = AutoModelForCausalLM.from_pretrained("/Users/lilyarunich/Documents/datascience/vvedenie_v_obr_est_yazika/proj/lerm") #путь до каталога с моделью
    #токенизируем текст
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+100,
        repetition_penalty=4.,
        temperature=0.5,
        num_beams=10,)
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    res = prefix + result
    return res

In [6]:
def generate_esenin(prefix):
    #загружаем модель
    model = AutoModelForCausalLM.from_pretrained("/Users/lilyarunich/Documents/datascience/vvedenie_v_obr_est_yazika/proj/esenin") #путь до каталога с моделью
    #токенизируем текст
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+100,
        repetition_penalty=4.,
        temperature=0.5,
        num_beams=10,)
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    res = prefix + result
    return res

In [7]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

In [8]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text

    if txt.lower() == 'привет':
        update.message.reply_text('и тебе привет! ')
    elif txt.lower() == 'хочу стих':
        update.message.reply_text('выбери автора: 1 - Лермонтов, 2 - Есенин')
        update.message.reply_text('введи запрос в формате: "_число_ _начало стихотворения_"')
    elif txt.split()[0] == '1':
        update.message.reply_text('ожидайте - началась генерация стихотворения Лермонтова ' + txt[2:])
        update.message.reply_text(generate_lerm(txt[2:]))
    elif txt.split()[0] == '2':
        update.message.reply_text('ожидайте - началась генерация стихотворения Есенина ' + txt[2:])
        update.message.reply_text(generate_esenin(txt[2:]))
    else:
        update.message.reply_text('если хотите начать генерацию стихотворения - введите "хочу стих" и следуйте инструкциям ')

In [9]:
updater = Updater("1797679219:AAFB8nPqDel_MhstnubHo0Qs1T9c_8yQ5Wo", use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

2021-08-31 14:36:20,097 - apscheduler.scheduler - INFO - Scheduler started
Distant resource does not have an ETag, we won't be able to reliably ensure reproducibility.
2021-08-31 14:37:20,737 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/configuration_utils.py", line 512, in get_config_dict
    resolved_config_file = cached_path(
  File "/usr/local/lib/python3.8/site-packages/transformers/file_utils.py", line 1370, in cached_path
    output_path = get_from_cache(
  File "/usr/local/lib/python3.8/site-packages/transformers/file_utils.py", line 1547, in get_from_cache
    raise OSError(
OSError: Distant resource does not have an ETag, we won't be able to reliably ensure reproducibility.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/telegram/ext/dis